# Imports

In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Bidirectional, LSTM, GRU, TimeDistributed, Activation, Flatten, Embedding, GlobalMaxPool1D, Dropout, Flatten, merge, RepeatVector, Permute, AveragePooling1D
from keras.optimizers import Adam



/Users/jmangia/anaconda2/envs/SwiftNLC/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Functions

In [2]:
# Prepare Glove File
def readGloveFile(gloveFile):
    with open(gloveFile, 'r') as f:
        wordToGlove = {}  # map from a token (word) to a Glove embedding vector
        wordToIndex = {}  # map from a token to an index
        indexToWord = {}  # map from an index to a token 

        for line in f:
            record = line.strip().split()
            token = record[0] # take the token (word) from the text line
            wordToGlove[token] = np.array(record[1:], dtype=np.float64) # associate the Glove embedding vector to a that token (word)

        tokens = sorted(wordToGlove.keys())
        for idx, tok in enumerate(tokens):
            kerasIdx = idx + 1  # 0 is reserved for masking in Keras (see above)
            wordToIndex[tok] = kerasIdx # associate an index to a token (word)
            indexToWord[kerasIdx] = tok # associate a word to a token (word). Note: inverse of dictionary above

    return wordToIndex, indexToWord, wordToGlove

In [3]:
# Create Pretrained Keras Embedding Weights Matrix
def createPretrainedEmbeddingMatrix(wordToGlove, wordToIndex):
    vocabLen = len(wordToIndex) + 1  # adding 1 to account for masking
    embDim = next(iter(wordToGlove.values())).shape[0]  # works with any glove dimensions (e.g. 50)

    embeddingMatrix = np.zeros((vocabLen, embDim))  # initialize with zeros
    for word, index in wordToIndex.items():
        embeddingMatrix[index, :] = wordToGlove[word] # create embedding: word index to Glove word embedding

    return vocabLen, embDim, embeddingMatrix

In [4]:
# Embedding
def getEncodedDocs(docs):
    encoded_docs = []

    tokenizer = RegexpTokenizer(r'\w+')
    for doc in docs:
        encoded_doc = []
        for word in tokenizer.tokenize(doc.lower()):
            index = wordToIndex[word]
            if index is not None:
                encoded_doc.append(index)
            else:
                encoded_doc.append(0)
        encoded_docs.append(encoded_doc)

    return encoded_docs

# Import GloVe Pretrained dataset and create Embedding Weight Matrix

In [5]:
max_length = 20
wordToIndex, indexToWord, wordToGlove = readGloveFile("glove/glove.6B.100d.txt")
vocabLen, embDim, embeddingMatrix = createPretrainedEmbeddingMatrix(wordToGlove, wordToIndex)

# Import intents file

In [6]:
import json
with open('PharmacyDataset.json') as json_data:
    intents = json.load(json_data)

# Padding, Encoding and Preparing final X and Y data for Training

In [7]:
classes = []
encodedUtterances = []

# loop through each sentence in our intents utterances
for intent in intents['intents']:
    classes.append(intent['intent'])
    encoded_docs = getEncodedDocs(intent['utterances'])
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    encodedUtterances.append(padded_docs)


In [8]:
currentClass = 0
train_x = []
train_y = []

for intent in classes:
    y = [0] * len(classes)
    y[currentClass] = 1

    for vector in encodedUtterances[currentClass]:
        train_x.append(vector)
        train_y.append(y)

    currentClass += 1

vectorSize = len(train_x[0])

train_X = np.array(train_x)
train_Y = np.array(train_y)

print(classes, "classes")
print(vectorSize, "vector size")
print(len(train_x), len(train_x[0]), "x")
print(len(train_y), len(train_y[0]), "y")
print(train_X.shape)
print(train_Y.shape)

([u'greeting', u'goodbye', u'thanks', u'options', u'adverse_drug', u'blood_pressure', u'blood_pressure_search', u'pharmacy_search'], 'classes')
(20, 'vector size')
(26, 20, 'x')
(26, 8, 'y')
(26, 20)
(26, 8)


# Prepare and Compile Keras / TensorFlow model

In [9]:
# model = Sequential([
#                    Embedding(vocabLen, embDim, weights=[embeddingMatrix], input_length=max_length, trainable=False),
#                    Bidirectional(LSTM(embDim, batch_size=1, input_shape=(None, embDim), return_sequences=True, dropout=0.0, recurrent_dropout=0.0)),
#                    GlobalMaxPool1D(),
#                    Dense(50, activation="relu"),
#                    Dense(len(train_y[0]), activation='softmax')
#                   ])

# input_ = Input(shape=(len(train_x[0]),), dtype='float')  #int32
# embedded = Embedding(vocabLen, embDim, weights=[embeddingMatrix], input_length=max_length, trainable=False)(input_)
# bidi = Bidirectional(LSTM(embDim, batch_size=1, input_shape=(None, embDim), return_sequences=True, dropout=0.0, recurrent_dropout=0.0))(embedded)
# gmax = GlobalMaxPool1D()(bidi)
# dense50 = Dense(50, activation="relu")(gmax)
# preds = Dense(len(train_y[0]), activation='softmax')(dense50)
# model = Model(input_, preds)

input_ = Input(shape=(len(train_x[0]),), dtype='float')  #int32
embedded = Embedding(vocabLen, embDim, weights=[embeddingMatrix], input_length=max_length, trainable=False)(input_)
#bidi = LSTM(embDim, batch_size=1, input_shape=(None, embDim), return_sequences=True, dropout=0.0, recurrent_dropout=0.0)(embedded)
bidi = Bidirectional(LSTM(embDim, batch_size=1, input_shape=(None, embDim), return_sequences=True, dropout=0.0, recurrent_dropout=0.0))(embedded)
# compute importance for each step
attention = Dense(1, activation='tanh')(bidi)
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
attention = RepeatVector(embDim*2)(attention)
attention = Permute([2, 1])(attention)
# merge
mer = merge([attention, bidi], "mul")
hid = AveragePooling1D(pool_length=len(train_x[0]))(mer)
hid = Flatten()(hid)
preds = Dense(len(train_y[0]), activation='softmax')(hid)
# Model
model = Model(input_, preds)


print("model - Bidirectional LSTM with Attention")
model.summary()


model - Bidirectional LSTM with Attention
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 100)      40000100    input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 20, 200)      160800      embedding_1[0][0]                
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 20, 1)        201         bidirectional_1[0][0]            
___________________________________________________________________

/Users/jmangia/anaconda2/envs/SwiftNLC/lib/python2.7/site-packages/ipykernel_launcher.py:28: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/jmangia/anaconda2/envs/SwiftNLC/lib/python2.7/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/jmangia/anaconda2/envs/SwiftNLC/lib/python2.7/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `AveragePooling1D` call to the Keras 2 API: `AveragePooling1D(pool_size=20)`


In [10]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model

In [11]:
model.fit(train_X, train_Y, epochs=250)

Epoch 1/250
26/26 [==============================] - 1s 28ms/step - loss: 2.0782 - acc: 0.1538
Epoch 2/250
26/26 [==============================] - 0s 849us/step - loss: 2.0767 - acc: 0.3077
Epoch 3/250
26/26 [==============================] - 0s 833us/step - loss: 2.0752 - acc: 0.3846
Epoch 4/250
26/26 [==============================] - 0s 824us/step - loss: 2.0734 - acc: 0.4231
Epoch 5/250
26/26 [==============================] - 0s 837us/step - loss: 2.0716 - acc: 0.5385
Epoch 6/250
26/26 [==============================] - 0s 833us/step - loss: 2.0695 - acc: 0.5385
Epoch 7/250
26/26 [==============================] - 0s 822us/step - loss: 2.0673 - acc: 0.5385
Epoch 8/250
26/26 [==============================] - 0s 888us/step - loss: 2.0650 - acc: 0.5000
Epoch 9/250
26/26 [==============================] - 0s 876us/step - loss: 2.0625 - acc: 0.5000
Epoch 10/250
26/26 [==============================] - 0s 944us/step - loss: 2.0598 - acc: 0.5000
Epoch 11/250
26/26 [====================

26/26 [==============================] - 0s 877us/step - loss: 1.4210 - acc: 0.7692
Epoch 87/250
26/26 [==============================] - 0s 920us/step - loss: 1.4122 - acc: 0.7692
Epoch 88/250
26/26 [==============================] - 0s 915us/step - loss: 1.4033 - acc: 0.7692
Epoch 89/250
26/26 [==============================] - 0s 921us/step - loss: 1.3940 - acc: 0.7692
Epoch 90/250
26/26 [==============================] - 0s 935us/step - loss: 1.3851 - acc: 0.7692
Epoch 91/250
26/26 [==============================] - 0s 958us/step - loss: 1.3759 - acc: 0.7692
Epoch 92/250
26/26 [==============================] - 0s 969us/step - loss: 1.3666 - acc: 0.7692
Epoch 93/250
26/26 [==============================] - 0s 905us/step - loss: 1.3580 - acc: 0.7692
Epoch 94/250
26/26 [==============================] - 0s 935us/step - loss: 1.3498 - acc: 0.7692
Epoch 95/250
26/26 [==============================] - 0s 940us/step - loss: 1.3417 - acc: 0.7692
Epoch 96/250
26/26 [=======================

Epoch 170/250
26/26 [==============================] - 0s 977us/step - loss: 0.9116 - acc: 0.8846
Epoch 171/250
26/26 [==============================] - 0s 933us/step - loss: 0.9073 - acc: 0.8846
Epoch 172/250
26/26 [==============================] - 0s 951us/step - loss: 0.9031 - acc: 0.8846
Epoch 173/250
26/26 [==============================] - 0s 954us/step - loss: 0.8989 - acc: 0.8846
Epoch 174/250
26/26 [==============================] - 0s 931us/step - loss: 0.8946 - acc: 0.8846
Epoch 175/250
26/26 [==============================] - 0s 926us/step - loss: 0.8904 - acc: 0.8846
Epoch 176/250
26/26 [==============================] - 0s 954us/step - loss: 0.8862 - acc: 0.8846
Epoch 177/250
26/26 [==============================] - 0s 960us/step - loss: 0.8821 - acc: 0.8846
Epoch 178/250
26/26 [==============================] - 0s 901us/step - loss: 0.8779 - acc: 0.9231
Epoch 179/250
26/26 [==============================] - 0s 949us/step - loss: 0.8738 - acc: 0.9231
Epoch 180/250
26/26 

# Save Model

In [12]:
model.save('SwiftNLCGloveRNN.h5')

# Test Model

In [13]:
# evaluate the model
loss, accuracy = model.evaluate(train_X, train_Y, verbose=0)
print('Loss: %f ' % (loss*100))
print('Accuracy: %f ' % (accuracy*100))

Loss: 62.928510 
Accuracy: 100.000000 


In [14]:
print(intents)

#test = np.array([[178126, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
#test_docs = getEncodedDocs(["Display blood values for patient"])
#test_docs = getEncodedDocs(["That was very useful"])
#test_docs = getEncodedDocs(["Show blood pressure results for patient"])
test_docs = getEncodedDocs(["Search for a pharmacy please"])



padded_test_docs = pad_sequences(test_docs, maxlen=max_length, padding='post')

print(padded_test_docs)

y_pred = model.predict(padded_test_docs)
print(y_pred)

max_value = max(y_pred[0])
print(max_value)

max_index = y_pred[0].tolist().index(max_value)
print(max_index)

print(intents['intents'][max_index]['intent'])

{u'intents': [{u'intent': u'greeting', u'utterances': [u'Hi', u'How are you', u'Is anyone there?', u'Hello', u'Good day']}, {u'intent': u'goodbye', u'utterances': [u'Bye', u'See you later', u'Goodbye']}, {u'intent': u'thanks', u'utterances': [u'Thanks', u'Thank you', u"That's helpful"]}, {u'intent': u'options', u'utterances': [u'How you could help me?', u'What you can do?', u'What help you provide?']}, {u'intent': u'adverse_drug', u'utterances': [u'How to check Adverse drug reaction?', u'List all drugs suitable for patient with adverse reaction', u'Which drugs dont have adverse reaction?']}, {u'intent': u'blood_pressure', u'utterances': [u'Open blood pressure module', u'I want to log blood pressure results', u'Blood pressure data management']}, {u'intent': u'blood_pressure_search', u'utterances': [u'I want to search for blood pressure result history', u'Show blood pressure results for patient', u'Find blood pressure results by ID']}, {u'intent': u'pharmacy_search', u'utterances': [u'Fi

# Export Word Embedding Array

In [15]:
for i in range(0,10):
    print(wordToIndex.keys()[i], wordToIndex[wordToIndex.keys()[i]], i)
    
import json
with open('Words.json', 'w') as fp:
    json.dump(wordToIndex, fp)


('biennials', 75684, 0)
('tripolitan', 365444, 1)
('biysk', 77319, 2)
('woode', 389559, 3)
('verplank', 377800, 4)
('mdbo', 239051, 5)
('sowell', 338477, 6)
('mdbu', 239054, 7)
('soestdijk', 336526, 8)
('spiders', 339422, 9)


# Export Model using CoreML Tools

In [16]:
import coremltools
coreml_model = coremltools.converters.keras.convert(model, input_names="vectors", output_names="entities")
coreml_model

ValueError: Keras layer '<class 'keras.legacy.layers.Merge'>' not supported. 

In [ ]:
coreml_model.save('SwiftNLCGloveRNN.mlmodel')